<a href="https://colab.research.google.com/github/lukeolson/imperial-multigrid/blob/master/lecture-1-mg-basics/7-multigrid-two-level-matrix-form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import scipy as sp
import numpy as np
import scipy.sparse as sparse
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
plt.style.use('seaborn-talk')

%matplotlib inline

In [2]:
def hnorm(r):
    """define ||r||_h = h ||r||_2"""
    n = len(r)
    h = 1.0 / (n+1)
    hrnorm = h * np.linalg.norm(r)
    return hrnorm

In [3]:
def poissonop(n):
    A = (n+1)**2 * sparse.diags([-1, 2, -1], [-1, 0, 1], shape=(n,n), format='csr')
    return A

def relax(A, u0, f, nu):
    u = u0.copy()
    n = A.shape[0]
    Dinv = 1.0 / (2.0 * (n+1)**2)
    omega = 2.0 / 3.0
    for steps in range(nu):
        u += omega * Dinv * (f - A * u)
    return u

def interpolation1d(nc, nf):
    d = np.repeat([[1, 2, 1]], nc, axis=0).T
    I = np.zeros((3,nc),dtype=int)
    for i in range(nc):
        I[:,i] = [2*i, 2*i+1, 2*i+2]
    J = np.repeat([np.arange(nc)], 3, axis=0)
    P = sparse.coo_matrix(
        (d.ravel(), (I.ravel(), J.ravel()))
        ).tocsr()
    return 0.5 * P

def twolevel(A, P, A1, u0, f0, nu):
    u0 = relax(A, u0, f0, nu) # pre-smooth
    f1 = P.T * (f0 - A * u0)  # restrict

    u1 = sla.spsolve(A1, f1)  # coarse solve

    u0 = u0 + P * u1          # interpolate
    u0 = relax(A, u0, f0, nu) # post-smooth
    return u0

In [4]:
# Problem setup
k = 9
n = 2**k - 1
nc = 2**(k-1) - 1
A = poissonop(n)
f = np.zeros(n)
u = np.random.rand(n)

# Multigrid Setup
P = interpolation1d(nc, n)
A1 = P.T * A * P

# Multigrid cycling
res = [hnorm(f - A * u)]
for i in range(10):
    u = twolevel(A, P, A1, u, f, 1)
    res.append(hnorm(f - A * u))

# Look at the residuals
res = np.array(res)
print(res[1:] / res[:-1])

[0.07289305 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111 0.11111111]


Is this to be expected?!